In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Conv1D, GlobalMaxPool1D, Input
from keras.layers import Dense, Dropout, MaxPooling1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split



data = pd.read_csv('drive/My Drive/ALDA/train.csv')

Using TensorFlow backend.


In [0]:
#print(data)
data = data.values
X_train = data[:,1]
Y_train = data[:,2]

print(Y_train)

[0 0 0 ... 0 0 0]


In [0]:
print(X_train)
print(Y_train)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
encoded = tokenizer.texts_to_sequences(X_train)

print(X_train[8])
print(encoded[8:9])

Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169
[[263, 22, 1, 286, 34847, 24, 1476, 2, 6, 489, 73, 14, 9215, 2, 330, 215, 10, 1, 23, 1473, 51, 47, 2898, 15, 35, 12, 199, 73, 1051, 1060, 9, 11, 16, 57, 1390, 37, 48, 40, 81, 11, 12, 480, 17, 5, 273, 7, 18, 55, 2, 1, 4850, 8153, 27, 26, 78, 669, 5, 1368, 11, 687, 2, 96147, 96148, 53, 8, 5, 702, 7774, 23, 9, 1094, 6, 46, 468, 41, 251, 16, 159, 324, 21, 3566, 10, 13824, 3439, 4584, 3890, 4851]]


In [0]:
max_len = 0;
for i in range(0, len(encoded)):
    if max_len < len(encoded[i]):
        max_len = len(encoded[i])

max_len = max_len + 1
print(max_len)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

1404
210338


In [0]:
from keras.preprocessing.sequence import pad_sequences

padded_docs = pad_sequences(encoded, maxlen=1404, padding='post')

print(padded_docs)
X_train, X_test, Y_train, Y_test = train_test_split( padded_docs, Y_train, test_size=0.2, random_state=42)

[[  688    75     1 ...     0     0     0]
 [96145    52  2635 ...     0     0     0]
 [  412   437    73 ...     0     0     0]
 ...
 [34278  7330  5208 ...     0     0     0]
 [    4    11   574 ...     0     0     0]
 [    4     7   134 ...     0     0     0]]


In [0]:
max_len = 0;
for i in range(0, len(X_train)):
    if max_len < len(X_train[i]):
        max_len = len(X_train[i])
        
print(max_len)
print(X_train.shape)

1404
(127656, 1404)


In [0]:
input_seq = Input(shape=(1404, ))
embed = Embedding(vocab_size, 100, input_length=1404)(input_seq)
x = Conv1D(256, 4, activation='relu')(embed)
x = MaxPooling1D(2, 1)(x)
x = Conv1D(128, 4, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
classi = Dense(1, activation='sigmoid')(x)
classifier = Model(input_seq, classi)
classifier.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
print(classifier.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1404)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1404, 100)         21033800  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1401, 256)         102656    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1400, 256)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1397, 128)         131200    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dens

In [0]:
classifier.fit(X_train, Y_train, batch_size=512, epochs = 30, 
    verbose=1, validation_data= (X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 [==============================] - 138s 1ms/step - loss: 0.2872 - acc: 0.9040 - val_loss: 0.2060 - val_acc: 0.9324
Epoch 2/30
127656/127656 [==============================] - 128s 1ms/step - loss: 0.1552 - acc: 0.9479 - val_loss: 0.1229 - val_acc: 0.9554
Epoch 3/30
127656/127656 [==============================] - 129s 1ms/step - loss: 0.1163 - acc: 0.9575 - val_loss: 0.1397 - val_acc: 0.9488
Epoch 4/30
127656/127656 [==============================] - 128s 1ms/step - loss: 0.1011 - acc: 0.9625 - val_loss: 0.1346 - val_acc: 0.9525
Epoch 5/30
127656/127656 [==============================] - 128s 1ms/step - loss: 0.0913 - acc: 0.9654 - val_loss: 0.1135 - val_acc: 0.9581
Epoch 6/30
127656/127656 [==============================] - 128s 1ms/step - loss: 0.0818 - acc: 0.9686 - val_loss: 0.1210 -

In [0]:
predictions = classifier.predict(X_test)
print(predictions)

for i in range(len(predictions)):
    if predictions[i]>0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0

print(predictions)

[[9.997331e-01]
 [0.000000e+00]
 [4.285574e-05]
 ...
 [2.778175e-06]
 [9.995764e-01]
 [7.258611e-09]]
[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

print(predictions)
Y_test = [[i] for i in Y_test]

print(Y_test)
print(classification_report(Y_test, predictions))

pd.DataFrame(
   confusion_matrix(Y_test, predictions),
   index = [['Actual', 'Actual'], ['Not Toxic', 'Toxic']],
   columns = [['Predicted', 'Predicted'], ['Not Toxic', 'Toxic']])

[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0

Predicted      
                 Not Toxic Toxic
Actual Not Toxic     27995   864
       Toxic           730  2326